# 🛍️ 產品推薦系統 - 使用您的資料

這個 Notebook 讓您上傳自己的資料檔案，執行完整的訓練和推薦流程。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ldsAS/product-recommendation-system/blob/main/notebooks/colab_demo.ipynb)

## 📋 需要準備的資料檔案

請準備以下三個 JSON Lines 格式的檔案：

1. **member** - 會員資料
2. **sales** - 銷售訂單
3. **salesdetails** - 訂單明細

### 資料格式說明

每個檔案都是 JSON Lines 格式（每行一個 JSON 物件）：

**member 範例**：
```json
{"會員編號":"CU000001","電話":"0937024682","總消費金額":17400,"累積紅利":500,"註冊日期":"2023-01-15"}
{"會員編號":"CU000002","電話":"0912345678","總消費金額":25600,"累積紅利":800,"註冊日期":"2023-02-20"}
```

**sales 範例**：
```json
{"訂單編號":"S000001","會員編號":"CU000001","訂單日期":"2024-01-10","訂單金額":1200,"門市代碼":"STORE01"}
{"訂單編號":"S000002","會員編號":"CU000001","訂單日期":"2024-01-15","訂單金額":800,"門市代碼":"STORE01"}
```

**salesdetails 範例**：
```json
{"訂單編號":"S000001","產品編號":"30463","數量":2,"單價":500,"小計":1000}
{"訂單編號":"S000001","產品編號":"31033","數量":1,"單價":200,"小計":200}
```

## 步驟 1: 環境設置

In [ ]:
%%capture
# 安裝依賴套件（隱藏輸出）
!pip install pandas numpy scikit-learn lightgbm pydantic pydantic-settings python-dotenv tqdm

In [ ]:
# 克隆專案
!git clone https://github.com/ldsAS/product-recommendation-system.git
%cd product-recommendation-system

# 設置環境
import os
import sys

project_path = '/content/product-recommendation-system'
os.chdir(project_path)
sys.path.insert(0, project_path)
os.environ['PYTHONPATH'] = project_path

print("✅ 環境設置完成！")
print(f"當前目錄: {os.getcwd()}")

## 步驟 2: 上傳資料檔案

請依序上傳三個資料檔案。

⚠️ **重要**: 檔案必須是 JSON Lines 格式（每行一個 JSON 物件）

In [ ]:
from google.colab import files
import shutil

print("📤 請上傳 member 檔案（會員資料）")
print("=" * 60)
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    shutil.move(filename, 'data/raw/member')
    print(f"✅ member 檔案已上傳: {len(uploaded[filename])} bytes")
else:
    print("❌ 未上傳檔案")

In [ ]:
print("📤 請上傳 sales 檔案（銷售訂單）")
print("=" * 60)
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    shutil.move(filename, 'data/raw/sales')
    print(f"✅ sales 檔案已上傳: {len(uploaded[filename])} bytes")
else:
    print("❌ 未上傳檔案")

In [ ]:
print("📤 請上傳 salesdetails 檔案（訂單明細）")
print("=" * 60)
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    shutil.move(filename, 'data/raw/salesdetails')
    print(f"✅ salesdetails 檔案已上傳: {len(uploaded[filename])} bytes")
else:
    print("❌ 未上傳檔案")

## 步驟 3: 驗證資料檔案

In [ ]:
import json
from pathlib import Path

print("🔍 驗證資料檔案...\n")

data_files = {
    'member': 'data/raw/member',
    'sales': 'data/raw/sales',
    'salesdetails': 'data/raw/salesdetails'
}

all_ok = True

for name, filepath in data_files.items():
    path = Path(filepath)
    
    if not path.exists():
        print(f"❌ {name}: 檔案不存在")
        all_ok = False
        continue
    
    # 讀取前幾行檢查格式
    try:
        with open(path, 'r', encoding='utf-8') as f:
            lines = [f.readline() for _ in range(3)]
            valid_lines = 0
            
            for line in lines:
                if line.strip():
                    json.loads(line)  # 驗證 JSON 格式
                    valid_lines += 1
        
        # 計算總行數
        with open(path, 'r', encoding='utf-8') as f:
            total_lines = sum(1 for line in f if line.strip())
        
        print(f"✅ {name}: {total_lines:,} 筆記錄，格式正確")
        
    except json.JSONDecodeError as e:
        print(f"❌ {name}: JSON 格式錯誤 - {e}")
        all_ok = False
    except Exception as e:
        print(f"❌ {name}: 讀取錯誤 - {e}")
        all_ok = False

print("\n" + "=" * 60)
if all_ok:
    print("✅ 所有資料檔案驗證通過！可以開始訓練。")
else:
    print("❌ 資料檔案驗證失敗，請檢查檔案格式。")
    print("\n💡 提示: 檔案必須是 JSON Lines 格式（每行一個 JSON 物件）")
print("=" * 60)

## 步驟 4: 訓練推薦模型

這個步驟會：
1. 載入並合併資料
2. 清理和驗證資料
3. 特徵工程
4. 訓練推薦模型
5. 評估模型效能

⏱️ 預計需要 2-5 分鐘（取決於資料量）

In [ ]:
print("🚀 開始訓練推薦模型...\n")
print("=" * 60)

# 使用 PYTHONPATH 執行訓練
!cd /content/product-recommendation-system && PYTHONPATH=/content/product-recommendation-system python src/train.py

print("\n" + "=" * 60)
print("✅ 訓練完成！")

## 步驟 5: 檢查訓練結果

In [ ]:
import os
from pathlib import Path

print("📊 訓練結果檢查\n")
print("=" * 60)

model_dir = Path('data/models')

if model_dir.exists():
    model_files = list(model_dir.glob('*.pkl'))
    
    if model_files:
        print(f"✅ 找到 {len(model_files)} 個模型檔案:\n")
        for model_file in model_files:
            size_mb = model_file.stat().st_size / (1024 * 1024)
            print(f"  📦 {model_file.name}")
            print(f"     大小: {size_mb:.2f} MB")
    else:
        print("❌ 未找到模型檔案")
        print("\n可能原因:")
        print("  1. 訓練過程出錯")
        print("  2. 資料格式不正確")
        print("  3. 資料量太少")
else:
    print("❌ 模型目錄不存在")

print("=" * 60)

## 步驟 6: 測試推薦功能

In [ ]:
# 確保路徑正確
import sys
import os

os.chdir('/content/product-recommendation-system')
if '/content/product-recommendation-system' not in sys.path:
    sys.path.insert(0, '/content/product-recommendation-system')

# 檢查模型是否存在
from pathlib import Path

model_dir = Path('data/models')
model_files = list(model_dir.glob('*.pkl'))

if not model_files:
    print("❌ 錯誤: 未找到模型檔案")
    print("請先執行步驟 4 訓練模型")
else:
    print(f"✅ 找到模型: {model_files[0].name}\n")
    
    # 導入模組
    from src.models.recommendation_engine import RecommendationEngine
    from src.models.data_models import RecommendationRequest
    
    # 初始化推薦引擎
    print("🔧 初始化推薦引擎...")
    engine = RecommendationEngine()
    print("✅ 推薦引擎已就緒\n")
    
    # 顯示模型資訊
    model_info = engine.get_model_info()
    print("📊 模型資訊:")
    print("=" * 60)
    print(f"模型版本: {model_info.model_version}")
    print(f"模型類型: {model_info.model_type}")
    print(f"訓練時間: {model_info.trained_at}")
    print(f"\n效能指標:")
    for key, value in model_info.metrics.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.4f}")
        else:
            print(f"  {key}: {value}")
    print("=" * 60)

## 步驟 7: 生成推薦

輸入會員資訊，獲取推薦結果

In [ ]:
# 創建測試請求
# 請根據您的實際資料修改以下參數

request = RecommendationRequest(
    member_code="CU000001",  # 修改為您的會員編號
    phone="0937024682",       # 修改為會員電話
    total_consumption=17400,  # 修改為總消費金額
    accumulated_bonus=500,    # 修改為累積紅利
    recent_purchases=[]       # 可選：最近購買的產品編號列表
)

print(f"🎯 為會員 {request.member_code} 生成推薦\n")
print(f"總消費: ${request.total_consumption:,}")
print(f"累積紅利: {request.accumulated_bonus} 點\n")

# 獲取推薦
try:
    recommendations = engine.get_recommendations(request)
    
    print("✨ 推薦結果:")
    print("=" * 60)
    
    for i, rec in enumerate(recommendations.recommendations, 1):
        print(f"\n{i}. 產品編號: {rec.product_id}")
        print(f"   信心分數: {rec.score:.2%}")
        print(f"   推薦理由: {rec.reason}")
        if rec.explanation:
            print(f"   詳細說明: {rec.explanation}")
    
    print("\n" + "=" * 60)
    print(f"回應時間: {recommendations.response_time_ms:.0f} ms")
    print("\n✅ 推薦成功！")
    
except Exception as e:
    print(f"\n❌ 推薦失敗: {e}")
    print("\n除錯資訊:")
    import traceback
    traceback.print_exc()

## 🎉 完成！

您已成功：
1. ✅ 上傳了資料檔案
2. ✅ 訓練了推薦模型
3. ✅ 生成了產品推薦

### 下一步

- 修改步驟 7 的會員資訊，測試不同會員的推薦
- 下載訓練好的模型檔案
- 查看完整文檔：[GitHub](https://github.com/ldsAS/product-recommendation-system)

### 下載模型

執行下方程式碼下載訓練好的模型：

In [ ]:
from google.colab import files
from pathlib import Path

model_dir = Path('data/models')
model_files = list(model_dir.glob('*.pkl'))

if model_files:
    for model_file in model_files:
        print(f"📥 下載 {model_file.name}...")
        files.download(str(model_file))
    print("✅ 下載完成！")
else:
    print("❌ 未找到模型檔案")